<a href="https://colab.research.google.com/github/mk7exe/millerite/blob/master/Phase3/Millerite_2D_CNN_V2_HP_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os
import h5py
# %tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# Load the TensorBoard notebook extension
%load_ext tensorboard

print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Downloading dataset files:

Local run: When run locally dataset files can be accessed from the project folder.
Colab run: Datasets should be downloaded from github (This downloads data sets in the notebook's path). We also mount google drive to load and save the model

In [0]:
local = False
if local:
    root_path = '/home/mk/Uni/GitHub Projects/millerite/Phase2/Steinhadt/datasets'
    drive_path = '/home/mk/GoogleDrive/Colab Notebooks'
    
else:
    ! wget -nv https://github.com/mk7exe/millerite/raw/master/Phase2/Steinhadt/datasets/data.zip -O data.zip
    root_path = './datasets'
    ! unzip -o data.zip
    # Mountng Google Drive
    # from google.colab import drive
    # drive.mount('/content/gdrive')
    # drive_path = 'gdrive/My Drive/Colab Notebooks'

In [0]:
def load_data():
    dataset = h5py.File(root_path + '/data_2D.h5', "r")
    xtrain = np.array(dataset["xtrain"][:])
    ytrain = np.array(dataset["ytrain"][:]) 
    xdev = np.array(dataset["xdev"][:])
    ydev = np.array(dataset["ydev"][:])
    xtest = np.array(dataset["xtest"][:])
    ytest = np.array(dataset["ytest"][:])
    
    surf_dataset = h5py.File(root_path + '/surface_2D.h5', "r")
    xsurf = np.array(surf_dataset["x"][:])
    ysurf = np.array(surf_dataset["y"][:])

    return xtrain, ytrain, xdev, ydev, xtest, ytest, xsurf, ysurf

xtrain, ytrain, xdev, ydev, xtest, ytest, xsurf, ysurf = load_data()

print(xtrain.shape, ytrain.shape, xdev.shape, ydev.shape, xtest.shape, 
      ytest.shape, xsurf.shape, ysurf.shape)
print(np.max(xtrain), np.max(xdev), np.max(xtest), np.max(xsurf))

In [0]:
ids = [id for id, elm in enumerate(ytrain) if elm < 2]
ytrain = ytrain[ids]
xtrain = xtrain[ids,:]
print(xtrain.shape, ytrain.shape)

In [0]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([512]))
HP_DROPOUT1 = hp.HParam('dropout1', hp.Discrete([0.0, 0.1, 0.2]))
HP_DROPOUT2 = hp.HParam('dropout2', hp.Discrete([0.0, 0.1, 0.2]))
HP_alpha = hp.HParam('alpha', hp.Discrete([0.0001, 0.001]))
HP_lambdaL2 = hp.HParam('lambdaL2', hp.Discrete([0.0, 0.001, 0.01]))
# HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['tanh', 'sigmoid', 'relu']))
# HP_OUT_ACTIVATION = hp.HParam('output', hp.Discrete(['linear']))

METRIC_ACCURACY = 'root_mean_squared_error'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT1, HP_DROPOUT2, HP_alpha, HP_lambdaL2],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='RMSE')]
  )

In [0]:
def train_test_model(hparams, epochs):
  lambdaL2 = hparams[HP_lambdaL2]
  dropout1 = hparams[HP_DROPOUT1]
  dropout2 = hparams[HP_DROPOUT2]
  learning_rate = hparams[HP_alpha]  
  num_units = hparams[HP_NUM_UNITS]
  activation = 'relu'
  out_activation = 'linear'


  model = keras.Sequential()
  model.add(keras.Input(shape=(100, 100, 1)))
  model.add(keras.layers.Conv2D(filters=16, kernel_size=(9, 9), 
                                activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

  model.add(keras.layers.Conv2D(filters=32, kernel_size=(5, 5),
                                activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

  model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3),
                                activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

  model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3),
                                activation=activation))
  model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dropout(dropout1))
  # model.add(keras.layers.GlobalMaxPooling2D())
  model.add(keras.layers.Dense(num_units, kernel_regularizer=
                                keras.regularizers.l2(lambdaL2)))
  model.add(keras.layers.Dropout(dropout2))
  model.add(keras.layers.Dense(num_units/4, kernel_regularizer=
                                keras.regularizers.l2(lambdaL2)))
  model.add(keras.layers.Dropout(dropout2))
  model.add(keras.layers.Dense(1, activation=out_activation))

  adam = keras.optimizers.Adam(learning_rate=learning_rate,
                               beta_1=0.9, beta_2=0.999) 
  model.compile(optimizer=adam, 
                loss=keras.losses.mean_squared_error, 
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
  model.fit(x_train, y_train, epochs=epochs, batch_size=32, verbose=0)

  _, train_accuracy = model.evaluate(x_train, y_train)
  _, accuracy = model.evaluate(x_dev, y_dev)
  return train_accuracy, accuracy

In [0]:
def run(run_dir, hparams, epochs):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    train_accuracy, accuracy = train_test_model(hparams, epochs)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [0]:
!rm -rf ./logs/*
%tensorboard --logdir ./logs/fit

In [0]:
epochs = 50
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout1 in HP_DROPOUT1.domain.values:
    for dropout2 in HP_DROPOUT2.domain.values:
      for alpha in HP_alpha.domain.values:
        for lambdaL2 in HP_lambdaL2.domain.values:
          hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT1: dropout1,
              HP_DROPOUT2: dropout1,
              HP_alpha: alpha,
              HP_lambdaL2: lambdaL2,
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/fit/' + run_name, hparams, epochs)
          session_num += 1